######  NB! This,InteractiveLoginAuthentication, is only needed to run 1st time, then when ws_config is written, use later CELL in notebook, that just reads that file
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
#sys.path.append(os.path.abspath("../azure-enterprise-scale-ml/esml/common/"))  # NOQA: E402
from esml import ESMLDataset, ESMLProject

p = ESMLProject()
#p.dev_test_prod="dev"
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)
######  NB!

In [ ]:
######  NB! This,InteractiveLoginAuthentication, is only needed to run 1st time, then when ws_config is written, use later CELL in notebook, that just reads that file
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from esml import ESMLDataset, ESMLProject

p = ESMLProject()
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)
######  NB!

In [ ]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p = ESMLProject()
p.active_model = 10 # Survived
p_factory = ESMLPipelineFactory(p, p.active_model["label"])

scoring_date = '2021-01-01 10:35:01.243860'
p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.

# IN_2_GOLD_SCORING
- Full ML-workflow: If you want to refine data from IN to GOLD, and SCORE model on GOLD, saves SCORED_GOLD in datalake
- Scenario: You want MLOps and full automation, ESMLPipelineFactory starting from Azure Data factory, and calling this genereated Azure ML Pipeline. 
    - Pipeline saving data automatically using the enterprise datalake/ESML AutoLake and ESML SDK

In [2]:
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_SCORING) # Creates pipeline from template

## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

# IN_2_GOLD
- If just wanting to refine data to GOLD, for a Power BI report (No ML involved)
- Scenario: You want to refine data from "IN_2_GOLD" with an easy way to READ/WRITE data (using the enterprise datalake via ESML AutoLake and ESML SDK)

In [ ]:
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD) # Creates pipeline from template

## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

# GOLD_SCORING
- If just wanting to SCORE, with best trained model in Azure ML Studio
- Scenario: Another service, Databricks or Azyre Synapse, refined data fron IN_2_GOLD", you just want to use Azure ML for scoring
    - Azure ML benefits:
        - Get lineage via Azure ML Datasets and Azure ML Model linage. 
        - Host the batch scoring pipleine in your Azure ML Studio
    - EMSL benefits
        - ESMLPipelinefactory + ESML AutoLake design, ESML extra logging & lineage.



In [ ]:
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.GOLD_SCORING) # Creates pipeline from template

## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

## 1b) When satisfied - `PUBLISH` pipeline (or rebuild and publish)

In [ ]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline, "_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint